In [1]:
from openai import OpenAI

client =  OpenAI(
    base_url = "http://localhost:11434/v1",
    api_key = "ollama"
)

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [ ]:
index_Settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_faq_questions" #must be lowercase

#create an index, only run one time as it is persistent
es_client.indices.create(index= index_name, body=index_Settings)


In [8]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
from tqdm.auto import tqdm

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [00:20<00:00, 45.23it/s]


In [13]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)     
    
    hits_results = []
    for hit in response['hits']['hits']:
        hits_results.append(hit['_source'])
    
    return hits_results

In [14]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
def llm_call(prompt):

    response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt,}],
    model="gemma2:2b",
)
    return response.choices[0].message.content

In [16]:
def rag_with_elastic(query):
    
    search_results = elastic_search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

In [18]:
query = "How do I Run Kafka for first time?"

In [19]:
rag_with_elastic(query)

This context describes how to create a Kafka Producer application, but it does not discuss installing Kafka or its deployment.  Let's break down the information provided:


**Understanding the Context:**

* **Focus on Kafka Producer Application:** This guide provides instructions for creating and running the `JsonProducer` Java code within the project directory. 
* **Java Implementation:** The context mentions building a Java application using the `JsonProducer.java` file.
* **Project Execution:** It calls for execution of a Java program: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`.   




**Reasons Why This Information Alone Is Insufficient to Answer "How do I Run Kafka for first time"**

*  Kafka is a distributed streaming platform, not just a Java application. To utilize its features, you need a more complete setup that involves:
    * **Installation:** Setting up the necessary software to handle Kafka's components. This includes